In [35]:
%load_ext autoreload
%autoreload 2

In [36]:
import os 
funcs = os.listdir('functions')

for func in funcs:
    if func.startswith('__') or func == 'import_all_functions.py':
        continue
    func = func.replace('.py','')
    print(f'{func} imported')
    exec(f'from functions.{func} import *')

import_to_df imported
add_had_remod_flag imported
get_kaggle_data imported
add_years_since imported
add_year_remod_pre1950_flag imported
quality_to_number imported
add_dummies_multiple_columns imported
add_value_dummies_multiple_columns imported
process_data imported
sum_onehots imported
add_zero_flag imported


In [60]:
import pandas as pd 
import pandas_profiling as pdp
import numpy as np
import pprint as pp
import seaborn as sns
from matplotlib import pyplot as plt
from catboost import Pool, CatBoostRegressor

from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [39]:
files = get_kaggle_data(
    competition="house-prices-advanced-regression-techniques"
    )

files downloaded for house-prices-advanced-regression-techniques competition
   test.csv
   data_description.txt
   train.csv
   sample_submission.csv
located in /competition_data/


In [40]:
input_data = import_to_df(files)

What files did we get?

In [41]:
input_data.keys()

dict_keys(['test', 'train', 'sample_submission'])

In [42]:
input_train_df = input_data['train']
input_test_df = input_data['test']

In [51]:
train_df = process_data(input_train_df)
target = 'SalePrice'
id_row = 'Id'
train_predictors = [
    c for c in train_df.columns 
    if target not in c
    if id_row not in c
]

In [52]:
test_df = process_data(input_test_df)
test_predictors = [
    c for c in test_df.columns 
    if target not in c
    if id_row not in c
]

Only keep columns in both dfs. Needed in case one hots generate unique columns

In [53]:
cols_to_use = list(set(train_predictors).intersection(test_predictors))

In [54]:
len(cols_to_use), len(train_predictors)

(299, 311)

Removed 1 column

In [72]:
X_train = train_df[cols_to_use]
y_train = train_df[target]

X_test = test_df[cols_to_use]
y_test_labels = test_df[id_row]

In [70]:
# initialize Pool
train_pool = Pool(X_train, y_train)
test_pool = Pool(X_test) 

param_dist = {
    'depth': [6,8,10],
    'learning_rate' : [0.01, 0.05, 0.1],
    'iterations'    : [30, 50, 100]
}
#Instantiate RandomSearchCV object
cbr = CatBoostRegressor()

random_grid = RandomizedSearchCV(cbr, param_dist, cv =5)

#Fit the model
random_grid.fit(X_train, y_train)

# Print the tuned parameters and score
print(random_grid.best_params_)
print(random_grid.best_score_)
# specify the training parameters 
# model = CatBoostRegressor(
#     iterations=5, 
#     depth=2, 
#     learning_rate=1, 
#     loss_function='RMSE'
# )
#train the model
# model.fit(train_pool)
# make the prediction using the resulting model
# preds = model.predict(test_pool)

rn: 32039.9469452	total: 296ms	remaining: 172ms
19:	learn: 31149.5784444	total: 314ms	remaining: 157ms
20:	learn: 30200.3276447	total: 334ms	remaining: 143ms
21:	learn: 29271.1078993	total: 351ms	remaining: 128ms
22:	learn: 28440.6989498	total: 366ms	remaining: 111ms
23:	learn: 27792.2170077	total: 382ms	remaining: 95.5ms
24:	learn: 27072.3377598	total: 397ms	remaining: 79.4ms
25:	learn: 26525.5007408	total: 412ms	remaining: 63.4ms
26:	learn: 26041.1314299	total: 427ms	remaining: 47.4ms
27:	learn: 25433.0977811	total: 442ms	remaining: 31.6ms
28:	learn: 24981.6446972	total: 457ms	remaining: 15.7ms
29:	learn: 24445.0113691	total: 473ms	remaining: 0us
0:	learn: 78344.6109856	total: 6.58ms	remaining: 191ms
1:	learn: 76050.5624874	total: 11.4ms	remaining: 159ms
2:	learn: 74092.9698826	total: 16.3ms	remaining: 147ms
3:	learn: 71903.7402277	total: 21.1ms	remaining: 137ms
4:	learn: 69817.1996084	total: 25.8ms	remaining: 129ms
5:	learn: 67875.0080707	total: 31.3ms	remaining: 125ms
6:	learn: 658

In [77]:
y_pred = random_grid.predict(X_test)

submission_df = pd.DataFrame()
submission_df[id_row] = y_test_labels
submission_df[target] = y_pred

In [78]:
submission_df.head()

,Id,SalePrice
0,1461,124061.734713
1,1462,156696.494915
2,1463,178224.018442
3,1464,180193.317506
4,1465,191269.212375


In [79]:
input_data['sample_submission'].head()

,Id,SalePrice
0,1461,169277.052498
1,1462,187758.393989
2,1463,183583.683570
3,1464,179317.477511
4,1465,150730.079977


In [80]:
submission_df.to_csv('submission1.csv', index=False)

In [83]:
!kaggle competitions submit -f 'submission1.csv' -m 'first attempt random hypers catboost' house-prices-advanced-regression-techniques

100%|██████████████████████████████████████| 33.7k/33.7k [00:02<00:00, 16.7kB/s]
Successfully submitted to House Prices - Advanced Regression Techniques

In [84]:
!kaggle competitions submissions -c house-prices-advanced-regression-techniques

fileName         date                 description                           status    publicScore  privateScore  
---------------  -------------------  ------------------------------------  --------  -----------  ------------  
submission1.csv  2021-05-03 11:30:27  first attempt random hypers catboost  complete  0.15473      None          
